# **Unraveling the Dynamics of Airfare Price Predictions**

Dennis Myasnyankin, Vannessa Salazar, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 599: Capstone Project

Professor Ebrahim Tarshizi

December 11, 2023

***

## **Data Importing**

### **Import Libraries**

In [1]:
import pandas as pd
import zipfile
import io
import os
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from pandas import json_normalize
from datetime import datetime
from io import BytesIO
from scipy import stats
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

### **Data Acquisition and Aggregation**

In [2]:
github_repo_url = 'https://github.com/VSbr22/Capstone/raw/main/data/jsons.zip'

# Fetch the zip file from the GitHub repository
response = requests.get(github_repo_url)
zip_file = BytesIO(response.content)

In [3]:
# Unzip the folder
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('your_unzipped_folder')

In [4]:
# List all JSON files in the 'jsons' subfolder
json_subfolder = os.path.join('your_unzipped_folder', 'jsons')
json_files = [file for file in os.listdir(json_subfolder) if file.endswith('.json')]

# Initialize an empty list for extracted data
out = []

# Iterate over each JSON file
for file_name in json_files:
    # Load JSON data
    with open(os.path.join(json_subfolder, file_name), 'r') as file:
        data = json.load(file)

    # Christines code for processing
    schedule_descs = data["groupedItineraryResponse"]["scheduleDescs"]
    leg_descs = data["groupedItineraryResponse"]["legDescs"]
    itinerary_groups = data["groupedItineraryResponse"]["itineraryGroups"]

    # Iterate over itinerary groups
    for group in itinerary_groups:
        itineraries = group.get("itineraries", [])
        departure_date = group["groupDescription"]["legDescriptions"][0]["departureDate"]
        departure_city = group["groupDescription"]["legDescriptions"][0]["departureLocation"]
        arrival_date = group["groupDescription"]["legDescriptions"][0].get("arrivalDate", departure_date)
        arrival_location = group["groupDescription"]["legDescriptions"][0]["arrivalLocation"]

        # Calculate "Days Until Departure" based on current date and departure date
        current_date = datetime.now()
        departure_date = datetime.strptime(departure_date, "%Y-%m-%d")
        days_until_departure = (departure_date - current_date).days

        # Iterate over itineraries in the group
        for itinerary in itineraries:
            legs = itinerary.get("legs", [])
            for leg in legs:
                schedules = leg_descs[leg.get('ref') - 1].get("schedules", [])
                for schedule in schedules:
                    schedule_data = schedule_descs[schedule.get('ref') - 1]

                    departure_time = schedule_data["departure"]["time"].split("+")[0]
                    arrival_time = schedule_data["arrival"]["time"].split("+")[0]
                    flight_duration = schedule_data["elapsedTime"]
                    operating_carrier = schedule_data["carrier"]["operating"]
                    aircraft_type = schedule_data["carrier"]["equipment"]["code"]
                    cabin_class = itinerary["pricingInformation"][0]["fare"]["passengerInfoList"][0]["passengerInfo"]["fareComponents"][0]["segments"][0]["segment"]["cabinCode"]
                    fare_class = itinerary["pricingInformation"][0]["fare"]["passengerInfoList"][0]["passengerInfo"]["fareComponents"][0]["segments"][0]["segment"]["bookingCode"]
                    price = itinerary["pricingInformation"][0]["fare"]["totalFare"]["totalPrice"]

                    # Add to output
                    out.append([
                        departure_city, departure_date, departure_time,
                        arrival_location, arrival_date, arrival_time,
                        flight_duration, operating_carrier, aircraft_type,
                        cabin_class, fare_class, price, days_until_departure
                    ])

# Creating a DataFrame
columns = [
    "Departure City", "Departure Date", "Departure Time",
    "Arrival Location", "Arrival Date", "Arrival Time",
    "Flight Duration", "Operating Carrier", "Aircraft Type",
    "Cabin Class", "Fare Class", "Price", "Days Until Departure"
]

df = pd.DataFrame(out, columns= columns)
df

,Departure City,Departure Date,Departure Time,Arrival Location,Arrival Date,Arrival Time,Flight Duration,Operating Carrier,Aircraft Type,Cabin Class,Fare Class,Price,Days Until Departure
0,CPH,2023-12-15,07:30:00,AAL,2023-12-15,08:15:00,45,DY,7M8,Y,Q,64.8,10
1,CPH,2023-12-15,10:45:00,AAL,2023-12-15,11:30:00,45,DY,7M8,Y,Q,64.8,10
2,CPH,2023-12-15,14:10:00,AAL,2023-12-15,14:55:00,45,DY,7M8,Y,Q,64.8,10
3,CPH,2023-12-15,18:55:00,AAL,2023-12-15,19:40:00,45,D8,73H,Y,Q,64.8,10
4,CPH,2023-12-15,20:20:00,AAL,2023-12-15,21:05:00,45,D8,73H,Y,Q,64.8,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8190,VIE,2023-11-22,07:00:00,ZRH,2023-11-22,08:25:00,85,OS,321,Y,M,1065.7,-13
8191,VIE,2023-11-22,09:45:00,ZRH,2023-11-22,11:10:00,85,LX,32Q,Y,M,1065.7,-13
8192,VIE,2023-11-22,09:45:00,ZRH,2023-11-22,11:10:00,85,LX,32Q,Y,M,1203.7,-13
8193,VIE,2023-11-22,07:00:00,ZRH,2023-11-22,08:25:00,85,OS,321,Y,M,1203.7,-13


***

## **Data Preprocessing**

### **Handling Missing Values**

In [22]:
# Check for missing values
df.isnull().sum()

Departure City          0
Departure Date          0
Departure Time          0
Arrival Location        0
Arrival Date            0
Arrival Time            0
Flight Duration         0
Operating Carrier       0
Aircraft Type           0
Cabin Class             0
Fare Class              0
Price                   0
Days Until Departure    0
dtype: int64

There is no missing data, so there is no need for imputation or removal.


In [23]:
# Count duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Remove duplicate rows (if any exist)
df.drop_duplicates(inplace = True)

Number of duplicate rows: 399


### **One Hot Encoding: Handling Categorical Features**

In [24]:
categorical_columns = ['Departure City', 'Operating Carrier', 'Aircraft Type', 'Cabin Class', 'Fare Class', 'Arrival Location']

df = pd.get_dummies(df, columns = categorical_columns)

### **Outliers: Remove Using Z-Score**

Rows from the original dataframe where all the z-scores are less than 3 are selected. Rows with z-scores greater than 3 are considered outliers and are excluded from the new dataframe.

In [25]:
z_scores = stats.zscore(df[numerical_columns])
df_no_outliers = df[(z_scores < 3)]

### **Convert to DateTime Format**

In [26]:
df['Departure Time'] = df['Departure Time'].str.replace('Z', '', regex = False)
df['Arrival Time'] = df['Arrival Time'].str.replace('Z', '', regex = False)

In [27]:
df['Departure Date'] = pd.to_datetime(df['Departure Date'], errors = 'coerce')
df['Departure Time'] = pd.to_datetime(df['Departure Time'] + '+0000', format = '%H:%M:%S%z', utc = True, errors = 'coerce')
df['Arrival Date'] = pd.to_datetime(df['Arrival Date'], errors = 'coerce')
df['Arrival Time'] = pd.to_datetime(df['Arrival Time'] + '+0000', format = '%H:%M:%S%z', utc = True, errors = 'coerce')

In [28]:
df.dtypes

Departure Date               datetime64[ns]
Departure Time          datetime64[ns, UTC]
Arrival Date                 datetime64[ns]
Arrival Time            datetime64[ns, UTC]
Flight Duration                       int64
                               ...         
Arrival Location_VIE                   bool
Arrival Location_YOW                   bool
Arrival Location_YUL                   bool
Arrival Location_YVR                   bool
Arrival Location_ZRH                   bool
Length: 316, dtype: object

In [29]:
# Convert datetime features to a numeric representation
df['Departure_date_numeric'] = (df['Departure Date'].astype('int64') // 10**9).astype('int32')
df['Arrival_date_numeric'] = (df['Arrival Date'].astype('int64') // 10**9).astype('int32')
df['Arrival_time_numeric'] = (df['Arrival Time'].astype('int64') // 10**9).astype('int32')
df['Departure_time_numeric'] = (df['Departure Time'].astype('int64') // 10**9).astype('int32')

In [30]:
print(df['Departure_date_numeric'])

0       1702598400
1       1702598400
2       1702598400
3       1702598400
4       1702598400
           ...    
8190    1700611200
8191    1700611200
8192    1700611200
8193    1700611200
8194    1700611200
Name: Departure_date_numeric, Length: 7796, dtype: int32


In [31]:
# Create a new dataframe that is a copy of the original dataframe
df2 = df.copy()

# Remove the original DateTime columns
columns_to_drop = ['Arrival Time', 'Arrival Date', 'Departure Date', 'Departure Time']
df2 = df2.drop(columns = columns_to_drop, axis = 1)
df2.head()

,Flight Duration,Price,Days Until Departure,Departure City_AMS,Departure City_BCN,Departure City_BKK,Departure City_BRU,Departure City_CDG,Departure City_CPH,Departure City_DMK,...,Arrival Location_USM,Arrival Location_VIE,Arrival Location_YOW,Arrival Location_YUL,Arrival Location_YVR,Arrival Location_ZRH,Departure_date_numeric,Arrival_date_numeric,Arrival_time_numeric,Departure_time_numeric
0,45,64.8,10,False,False,False,False,False,True,False,...,False,False,False,False,False,False,1702598400,1702598400,2086008196,2086005496
1,45,64.8,10,False,False,False,False,False,True,False,...,False,False,False,False,False,False,1702598400,1702598400,2086019896,2086017196
2,45,64.8,10,False,False,False,False,False,True,False,...,False,False,False,False,False,False,1702598400,1702598400,2086032196,2086029496
3,45,64.8,10,False,False,False,False,False,True,False,...,False,False,False,False,False,False,1702598400,1702598400,2086049296,2086046596
4,45,64.8,10,False,False,False,False,False,True,False,...,False,False,False,False,False,False,1702598400,1702598400,2086054396,2086051696


In [32]:
df2.dtypes

Flight Duration             int64
Price                     float64
Days Until Departure        int64
Departure City_AMS           bool
Departure City_BCN           bool
                           ...   
Arrival Location_ZRH         bool
Departure_date_numeric      int32
Arrival_date_numeric        int32
Arrival_time_numeric        int32
Departure_time_numeric      int32
Length: 316, dtype: object